# Histogrammar basic tutorial

Histogrammar is a Python package that allows you to make histograms from numpy arrays, and pandas and spark dataframes. (There is also a scala backend for Histogrammar.) 

This basic tutorial shows how to:
- make histograms with numpy arrays and pandas dataframes, 
- plot them, 
- make multi-dimensional histograms,
- the various histogram types,
- to make many histograms at ones,
- and store and retrieve them. 

Enjoy!

In [ ]:
%%capture
# install histogrammar (if not installed yet)

!"{sys.executable}" -m pip install histogrammar

In [ ]:
import histogrammar as hg

In [ ]:
import numpy as np
import pandas as pd

## Data generation
Let's first load some data!

In [ ]:
# open a pandas dataframe for use below
from histogrammar import resources

df = pd.read_csv(resources.data("test.csv.gz"), parse_dates=["date"])

In [ ]:
df.head()

## Let's fill a histogram!

Histogrammar treats histograms as objects. You will see this has various advantages.

Let's fill a simple histogram with a numpy array.

In [ ]:
# this creates a histogram with 100 even-sized bins in the (closed) range [-5, 5]
hist1 = hg.Bin(num=100, low=-5, high=5)

In [ ]:
# filling it with one data point:
hist1.fill(0.5)

In [ ]:
hist1.entries

In [ ]:
# filling the histogram with an array:
hist1.fill.numpy(np.random.normal(size=10000))

In [ ]:
hist1.entries

In [ ]:
# let's plot it
hist1.plot.matplotlib();

In [ ]:
# Alternatively, you can call this to make the same histogram:
# hist1 = hg.Histogram(num=100, low=-5, high=5)

Histogrammar also supports open-ended histograms, which are sparsely represented. Open-ended histograms are used when you have a distribution of known scale (bin width) but unknown domain (lowest and highest bin index). Bins in a sparse histogram only get created and filled if the corresponding data points are encountered. 

A sparse histogram has a `binWidth`, and optionally an `origin` parameter. The `origin` is the left edge of the bin whose index is 0 and is set to 0.0 by default. Sparse histograms are nice if you don't want to restrict the range, for example for tracking data distributions over time, which may have large, sudden outliers.

In [ ]:
hist2 = hg.SparselyBin(binWidth=10, origin=0)

In [ ]:
hist2.fill.numpy(df['age'].values)

In [ ]:
hist2.plot.matplotlib();

In [ ]:
# Alternatively, you can call this to make the same histogram:
# hist2 = hg.SparselyHistogram(binWidth=10)

## Filling from a dataframe

Let's make the same 1d (sparse) histogram directly from a (pandas) dataframe.

In [ ]:
hist3 = hg.SparselyBin(binWidth=10, origin=0, quantity='age')
hist3.fill.numpy(df)
hist3.plot.matplotlib();

When importing histogrammar, pandas (and spark) dataframes get extra functions to create histograms that all start with "hg_". For example: hg_Bin or hg_SparselyBin.
Note that the column "age" is picked by setting quantity="age", and also that the filling step is done automatically.

In [ ]:
# Alternatively, do:
hist3 = df.hg_SparselyBin(binWidth=10, origin=0, quantity='age')

# ... where hist3 automatically picks up column age from the dataframe,
# ... and does not need to be filled by calling fill.numpy() explicitly.

### Handy histogram methods

For any 1-dimensional histogram extract the bin entries, edges and centers as follows:

In [ ]:
# full range of bin entries, and those in a specified range:
(hist3.bin_entries(), hist3.bin_entries(low=30, high=80))

In [ ]:
# full range of bin edges, and those in a specified range:
(hist3.bin_edges(), hist3.bin_edges(low=31, high=71))

In [ ]:
# full range of bin centers, and those in a specified range:
(hist3.bin_centers(), hist3.bin_centers(low=31, high=80))

In [ ]:
hsum = hist2 + hist3
hsum.entries

In [ ]:
hsum *= 4
hsum.entries

## Irregular bin histogram variants

There are two other open-ended histogram variants in addition to the SparselyBin we have seen before. Whereas SparselyBin is used when bins have equal width, the others offer similar alternatives to a single fixed bin width.

There are two ways:
- CentrallyBin histograms, defined by specifying bin centers;
- IrregularlyBin histograms, with irregular bin edges.

They both partition a space into irregular subdomains with no gaps and no overlaps.

In [ ]:
hist4 = hg.CentrallyBin(centers=[15, 25, 35, 45, 55, 65, 75, 85, 95], quantity='age')
hist4.fill.numpy(df)
hist4.plot.matplotlib();

In [ ]:
hist4.bin_edges()

Note the slightly different plotting style for CentrallyBin histograms (e.g. x-axis labels are central values instead of edges).

## Multi-dimensional histograms

Let's make a multi-dimensional histogram. In Histogrammar, a multi-dimensional histogram is composed as two recursive histograms. 

We will use histograms with irregular binning in this example.

In [ ]:
edges1 = [-100, -75, -50, -25, 0, 25, 50, 75, 100]
edges2 = [-200, -150, -100, -50, 0, 50, 100, 150, 200]

In [ ]:
hist1 = hg.IrregularlyBin(edges=edges1, quantity='latitude')
hist2 = hg.IrregularlyBin(edges=edges2, quantity='longitude', value=hist1)

# for 3 dimensions or higher simply add the 2-dim histogram to the value argument
hist3 = hg.SparselyBin(binWidth=10, quantity='age', value=hist2)

In [ ]:
hist1.bin_centers()

In [ ]:
hist2.bin_centers()

In [ ]:
hist2.fill.numpy(df)
hist2.plot.matplotlib();

In [ ]:
# number of dimensions per histogram
(hist1.n_dim, hist2.n_dim, hist3.n_dim)

### Accessing bin entries

For most 2+ dimensional histograms, one can get the bin entries and centers as follows:

In [ ]:
from histogrammar.plot.hist_numpy import get_2dgrid

x_labels, y_labels, grid = get_2dgrid(hist2)

In [ ]:
y_labels, grid

### Accessing a sub-histogram

Depending on the histogram type of the first axis, hg.Bin or other, one can access the sub-histograms directly from:
hist.values or 
hist.bins

In [ ]:
# Acces sub-histograms from IrregularlyBin from hist.bins
# The first item of the tuple is the lower bin-edge of the bin.
hist2.bins[1]

In [ ]:
h = hist2.bins[1][1]
h.plot.matplotlib()

In [ ]:
h.bin_entries()

## Histogram types recap

So far we have covered the histogram types: 
- Bin histograms: with a fixed range and even-sized bins,
- SparselyBin histograms: open-ended and with a fixed bin-width,
- CentrallyBin histograms: open-ended and using bin centers.
- IrregularlyBin histograms: open-ended and using (irregular) bin edges,

All of these process numeric variables only.

### Categorical variables

For categorical variables use the Categorize histogram
- Categorize histograms: accepting categorical variables such as strings and booleans.



In [ ]:
histy = hg.Categorize('eyeColor')
histx = hg.Categorize('favoriteFruit', value=histy)

In [ ]:
histx.fill.numpy(df)
histx.plot.matplotlib();

In [ ]:
# show the datatype(s) of the histogram
histx.datatype

Categorize histograms also accept booleans:

In [ ]:
histy = hg.Categorize('isActive')
histy.fill.numpy(df)
histy.plot.matplotlib();

In [ ]:
histy.bin_entries()

In [ ]:
histy.bin_labels()
# histy.bin_centers() will work as well for Categorize histograms

### Other histogram functionality

There are several more histogram types:
- Minimize, Maximize: keep track of the min or max value of a numeric distribution,
- Average, Deviate: keep track of the mean or mean and standard deviation of a numeric distribution,
- Sum: keep track of the sum of a numeric distribution,
- Stack: keep track how many data points pass certain thresholds.
- Bag: works like a dict, it keeps tracks of all unique values encountered in a column, and can also do this for vector s of numbers. For strings, Bag works just like the Categorize histogram.

In [ ]:
hmin = df.hg_Minimize('latitude')
hmax = df.hg_Maximize('longitude')
(hmin.min, hmax.max)

In [ ]:
havg = df.hg_Average('latitude')
hdev = df.hg_Deviate('longitude')
(havg.mean, hdev.mean, hdev.variance)

In [ ]:
hsum = df.hg_Sum('age')
hsum.sum

In [ ]:
# let's illustrate the Stack histogram with longitude distribution
# first we plot the regular distribution
hl = df.hg_SparselyBin(25, 'longitude')
hl.plot.matplotlib();

In [ ]:
# Stack counts how often data points are greater or equal to the provided thresholds
thresholds = [-200, -150, -100, -50, 0, 50, 100, 150, 200]

In [ ]:
hs = df.hg_Stack(thresholds=thresholds, quantity='longitude')
hs.thresholds

In [ ]:
hs.bin_entries()

Stack histograms are useful to make efficiency curves.

With all these histograms you can make multi-dimensional histograms. For example, you can evaluate the mean and standard deviation of one feature as a function of bins of another feature. (A "profile" plot, similar to a box plot.) 

In [ ]:
hav = hg.Deviate('age')
hlo = hg.SparselyBin(25, 'longitude', value=hav)
hlo.fill.numpy(df)

In [ ]:
hlo.bins

In [ ]:
hlo.plot.matplotlib();

### Convenience functions

There are several convenience functions to make such composed histograms. These are:
- [Profile](https://histogrammar.github.io/histogrammar-docs/specification/1.0/#profile): Convenience function for creating binwise averages.
- [SparselyProfile](https://histogrammar.github.io/histogrammar-docs/specification/1.0/#sparselyprofile): Convenience function for creating sparsely binned binwise averages.
- [ProfileErr](https://histogrammar.github.io/histogrammar-docs/specification/1.0/#profileerr): Convenience function for creating binwise averages and variances.
- [SparselyProfile](https://histogrammar.github.io/histogrammar-docs/specification/1.0/#sparselyprofileerr): Convenience function for creating sparsely binned binwise averages and variances.
- [TwoDimensionallyHistogram](https://histogrammar.github.io/histogrammar-docs/specification/1.0/#twodimensionallyhistogram): Convenience function for creating a conventional, two-dimensional histogram.
- [TwoDimensionallySparselyHistogram](https://histogrammar.github.io/histogrammar-docs/specification/1.0/#twodimensionallysparselyhistogram): Convenience function for creating a sparsely binned, two-dimensional histogram.

In [ ]:
# For example, call this convenience function to make the same histogram as above:
hlo = df.hg_SparselyProfileErr(25, 'longitude', 'age')
hlo.plot.matplotlib();

### Overview of histograms

Here you can find the list of all available histograms and aggregators and how to use each one: 

https://histogrammar.github.io/histogrammar-docs/specification/1.0/

The most useful aggregators are the following. Tinker with them to get familiar; building up an analysis is easier when you know "there's an app for that."

**Simple counters:**

  * [`Count`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#count-sum-of-weights): just counts. Every aggregator has an `entries` field, but `Count` _only_ has this field.
  * [`Average`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#average-mean-of-a-quantity) and [`Deviate`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#deviate-mean-and-variance): add mean and variance, cumulatively.
  * [`Minimize`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#minimize-minimum-value) and [`Maximize`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#maximize-maximum-value): lowest and highest value seen.

**Histogram-like objects:**

  * [`Bin`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#bin-regular-binning-for-histograms) and [`SparselyBin`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#sparselybin-ignore-zeros): split a numerical domain into uniform bins and redirect aggregation into those bins.
  * [`Categorize`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#categorize-string-valued-bins-bar-charts): split a string-valued domain by unique values; good for making bar charts (which are histograms with a string-valued axis).
  * [`CentrallyBin`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#centrallybin-fully-partitioning-with-centers) and [`IrregularlyBin`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#irregularlybin-fully-partitioning-with-edges): split a numerical domain into arbitrary subintervals, usually for separate plots like particle pseudorapidity or collision centrality.

**Collections:**

  * [`Label`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#label-directory-with-string-based-keys), [`UntypedLabel`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#untypedlabel-directory-of-different-types), and [`Index`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#index-list-with-integer-keys): bundle objects with string-based keys (`Label` and `UntypedLabel`) or simply an ordered array (effectively, integer-based keys) consisting of a single type (`Label` and `Index`) or any types (`UntypedLabel`).
  * [`Branch`](
https://histogrammar.github.io/histogrammar-docs/specification/1.0/#branch-tuple-of-different-types): for the fourth case, an ordered array of any types. A `Branch` is useful as a "cable splitter". For instance, to make a histogram that tracks minimum and maximum value, do this:





## Making many histograms at once

There a nice method to make many histograms in one go. See here.

By default automagical binning is applied to make the histograms.

More details one how to use this function are found in in the advanced tutorial.

In [ ]:
hists = df.hg_make_histograms()

In [ ]:
hists.keys()

In [ ]:
h = hists['transaction']
h.plot.matplotlib();

In [ ]:
h = hists['date']
h.plot.matplotlib();

In [ ]:
# you can also select which and make multi-dimensional histograms
hists = df.hg_make_histograms(features = ['longitude:age'])

In [ ]:
hist = hists['longitude:age']
hist.plot.matplotlib();

## Storage

Histograms can be easily stored and retrieved in/from the json format.

In [ ]:
# storage
hist.toJsonFile('long_age.json')

In [ ]:
# retrieval
factory = hg.Factory()
hist2 = factory.fromJsonFile('long_age.json')
hist2.plot.matplotlib();

In [ ]:
# we can store the histograms if we want to
import json

from histogrammar.util import dumper

# store
with open('histograms.json', 'w') as outfile:
    json.dump(hists, outfile, default=dumper)

# and load again
with open('histograms.json') as handle:
    hists2 = json.load(handle)

In [ ]:
hists.keys()

## Advanced tutorial

The [advanced tutorial](https://nbviewer.jupyter.org/github/histogrammar/histogrammar-python/blob/master/histogrammar/notebooks/histogrammar_tutorial_advanced.ipynb) shows:
- How to work with spark dataframes.
- More details on this nice method to make many histograms in one go. For example how to set bin specifications.
